# Build UNet based Autoencoder with Further Options

# Test GPU (Optional)
Before Starting, kindly check the available GPU from the Google Server, GPU model and other related information. It might help!

In [ ]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))
print("Is GPU Initialized yet?", torch.cuda.is_initialized())

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/GDrive')

Mounted at /content/GDrive


Move to the Target Directory

In [ ]:
%cd /content/GDrive/MyDrive

/content/GDrive/MyDrive/Colab_Notebooks/Research/PPG2ABP


List the Files and Folders Located in the Current Directory

In [ ]:
!ls

## Evaluation of Predicting BP from PPG and ECG



#Import Libraries

In [ ]:
import os
import h5py
import scipy
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, concatenate, BatchNormalization, Activation, add
from keras.layers import Conv2D, MaxPooling2D, Reshape, Flatten, Dense
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from predict_test import predict_test_data
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from UNet_1DCNN import UNet
from pathlib import Path
from Prepare_Train_Dictionary import prepareTrainDict
%matplotlib inline
sns.set_style('white')

# Import Data Here and Process

# Build UNet based Autoencoder

Configurations

In [ ]:
num_channel = 4       # 1 to 4
feature_number = 1024 # Number of Features to be Extracted
signal_length = 1024  # Signal Length
model_depth = 5       # Number of layers in the Model
model_width = 128     # Number of Filters or Kernels in the input layer of the model
kernel_size = 3       # Size of the Kernel throughout the network
D_S = 0               # Use Deep Supervision or not (0: No, 1: Yes)
A_E = 1               # Use the Segmentation Model UNet as an Autoencoder to Extract Features (0: No, 1: Yes)

Prepare Data for Deep Supervision (if used)

In [ ]:
def prepareTrainDict(y, model_depth, signal_length):
  def approximate(inp, w_len, signal_length):
    op = np.zeros((len(inp),signal_length//w_len))
    for i in range(0,signal_length,w_len):
      try:
        op[:,i//w_len] = np.mean(inp[:,i:i+w_len],axis=1)
      except Exception as e:
        print(e)
        print(i)
  	
    return op

  out = {}
  Y_Train_dict = {}
  out['out'] = np.array(y)
  Y_Train_dict['out'] = out['out']
  for i in range(1, (model_depth+1)):
    name = f'level{i}'
    out[name] = np.expand_dims(approximate(np.squeeze(y), 2**i, signal_length),axis = 2)
    Y_Train_dict[f'level{i}'] = out[f'level{i}']
  
  return out, Y_Train_dict

Create the Model based on the Configurations set prior

In [ ]:
# Build model for PPG2BP - Shallow-Wide U-net as AutoEncoder
Model = UNet(signal_length, model_depth, num_channel, model_width, kernel_size, D_S, A_E, feature_number)
Model.compile(optimizer='adam', loss='mse', metrics=['mae'])

Train Model

In [ ]:
if D_S == 1: # AutoEncoder with Deep Supervision
  # Prepare Dictionary for Parameters for Deep Supervision
  X_Train1 = X_Train
  X_Test1 = X_Test
  X_Val1 = X_Val
  [Y_Train1, Y_Train_dict] = prepareTrainDict(Y_Train, model_depth, signal_length)
  [Y_Test1, Y_Test_dict] = prepareTrainDict(Y_Test, model_depth, signal_length)
  [Y_Val1, Y_Val_dict] = prepareTrainDict(Y_Val, model_depth, signal_length)
  # Create Losses for D_S Levels
  loss_weights = np.zeros(PPG2BP_model_depth)

  for i in range(0, PPG2BP_model_depth):
    loss_weights[i] = 1-(i*0.1)
   
  loss_weights
  # Train Network
  callbacks = [EarlyStopping(monitor='val_out_loss', patience=20, mode='min'), ModelCheckpoint('Result.h5', verbose=1, monitor='val_out_loss', save_best_only=True, mode='min')]
  Model.fit(X_Train1, Y_Train_dict, epochs=200, batch_size= 128, verbose=1, validation_data= (X_Val1, Y_Val_dict), shuffle= True, callbacks= callbacks)

elif D_S == 0: # AutoEncoder without Deep Supervision
  callbacks = [EarlyStopping(monitor='val_loss', patience=20, mode='min'), ModelCheckpoint('Result.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
  Model.fit(X_Train, Y_Train, epochs= 200, batch_size= 128, verbose=1, validation_data= (X_Val, Y_Val), shuffle= True, callbacks= callbacks)

# Validation Data is set to independent here by default. Use "Validation_split" otherwise.

Extrac Feature from the AutoEncoder

In [ ]:
Feature_Extractor = Model(inputs = Model.input, outputs = Model.get_layer('features').output)
#
Train_Features = Feature_Extractor.predict(X_Train1)
Test_Features = Feature_Extractor.predict(X_Test1)